## Grafiquitos

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pandas as pd
import seaborn as sns

In [ ]:
BENCH_DIR = '../../bench/'
GRAPH_DIR = '../../graph/'

CSV_COLUMNS = ["gyms","stops","bagSize","time","reps","status",
               "generator","result","algorithm","subtype","seed1","seed2"]

Configuración de matplotlib

In [ ]:
def plt_setlegend(ax, legend=None):
    if legend is not None:
        l = ax.legend(legend, loc='best', frameon=True, borderpad=0.5)
    else:
        l = ax.legend(loc='best', frameon=True, borderpad=0.5)
    l.get_frame().set_linewidth(1)

### Leemos los benchmarks
Separamos los de time de los de result, ya que los últimos se corren una sola vez y pueden tener mucho ruido

In [ ]:
def readDf(wildcard):
    files = glob.glob(os.path.join(BENCH_DIR, wildcard))
    dfs = (pd.read_csv(f, names=CSV_COLUMNS, header=None) for f in files)
    df = pd.concat(dfs, ignore_index=True)
    df["nodes"] = df["gyms"] + df["stops"]
    df["time"] = df["time"] / 1e9
    return df

df_time = readDf("*-time-*")
df_result = readDf("*-result-*")

### Datos de las mediciones

In [ ]:
df_time[["algorithm","subtype"]].drop_duplicates()

In [ ]:
df_time.describe()

In [ ]:
df_result.describe()

### Graficos de tiempo

In [ ]:
is_exacto = df_time["algorithm"] == 'exacto'
is_greedy = df_time["algorithm"] == 'greedy'
is_local = df_time["algorithm"] == 'local'

is_bruteforce = df_time["subtype"] == 'none'
is_backtracking = df_time["subtype"] == 'backtracking'
is_2opt = df_time["subtype"] == 'dos_opt'
is_swap = df_time["subtype"] == 'swap'

In [ ]:
plt.clf()
fig, ax = plt.subplots()

df_time[is_exacto & is_bruteforce].plot(x="nodes", y="time", ax=ax,
                                        label="fuerza bruta", style='o')

df_time[is_exacto & is_backtracking].plot(x="nodes", y="time", ax=ax,
                                        label="backtracking", style='o')
plt.ylabel('Tiempos en segundos')
plt_setlegend(ax)
plt.savefig(GRAPH_DIR + 'time-exacto.png')
plt.show()

In [ ]:
plt.clf()
fig, ax = plt.subplots()

df_time[is_greedy].plot(x="nodes", y="time", ax=ax,
                        label="heurística greedy", style='o')
plt.ylabel('Tiempos en segundos')
plt_setlegend(ax)
plt.savefig(GRAPH_DIR + 'time-greedy.png')
plt.show()

In [ ]:
plt.clf()
fig, ax = plt.subplots()

ax = df_time[is_local & is_2opt].plot(x="nodes", y="time", ax=ax,
                                 label="heurística local, 2opt", style='o')
ax = df_time[is_local & is_swap].plot(x="nodes", y="time", ax=ax,
                                 label="heurística local, swap", style='o')
plt.ylabel('Tiempos en segundos')
plt_setlegend(ax)
plt.savefig(GRAPH_DIR + 'time-local.png')
plt.show()

### Graficos de precisión

In [ ]:
is_exacto = df_result["algorithm"] == 'exacto'
is_greedy = df_result["algorithm"] == 'greedy'
is_local = df_result["algorithm"] == 'local'

is_2opt = df_result["subtype"] == 'dos_opt'
is_swap = df_result["subtype"] == 'swap'

merge_cols = ["gyms","stops","nodes","bagSize","generator","seed1","seed2"]
columns = merge_cols + ["result"]
ext_columns = columns + ["algorithm", "subtype"]

small_exacto = df_result[is_exacto][columns]
small_exacto = small_exacto[small_exacto["result"] != np.inf]
small_exacto.rename(columns={"result": "result_exacto"}, inplace=True)

def getPrecision(data_filter, name):
    tmp = pd.merge(df_result[data_filter][ext_columns],
                           small_exacto, on=merge_cols)
    tmp = tmp[tmp["result"] != np.inf]
    tmp["precision"] = tmp["result"] / tmp["result_exacto"]
    tmp["name"] = name
    return tmp

prec_greedy = getPrecision(is_greedy, 'greedy')
prec_local_2opt = getPrecision(is_local & is_2opt, 'local_2opt')
prec_local_swap = getPrecision(is_local & is_swap, 'local_swap')

prec_all = prec_greedy.append(prec_local_2opt).append(prec_local_swap)

In [ ]:
sns.boxplot(x='name', y='precision', data=prec_all)
plt.savefig(GRAPH_DIR + 'precision-small-all.png')
plt.show()

In [ ]:
sns.boxplot(x='nodes', y='precision', data=prec_local_2opt)
plt.savefig(GRAPH_DIR + 'precision-small-local-2opt.png')
plt.show()

In [ ]:
sns.boxplot(x='nodes', y='precision', data=prec_local_swap)
plt.savefig(GRAPH_DIR + 'precision-small-local-swap.png')
plt.show()